In [5]:
import cv2
import sys
import logging
from typing import Optional

# Configuración del logging para monitoreo y depuración
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class CameraStreamer:
    """Clase para manejar la captura y visualización de video usando OpenCV y GStreamer."""

    def __init__(self, camera_index: int = 0, width: int = 1280, height: int = 720, framerate: int = 30):
        """
        Inicializa el streamer de la cámara.

        Args:
            camera_index (int): Índice de la cámara (default: 0).
            width (int): Ancho de la resolución (default: 1280).
            height (int): Alto de la resolución (default: 720).
            framerate (int): Tasa de frames por segundo (default: 30).
        """
        self.camera_index = camera_index
        self.width = width
        self.height = height
        self.framerate = framerate
        self.cap: Optional[cv2.VideoCapture] = None
        self.window_name = "Camera Feed"

    def create_gstreamer_pipeline(self) -> str:
        """
        Crea el pipeline de GStreamer para capturar video desde la cámara.

        Returns:
            str: Cadena del pipeline de GStreamer.
        """
        pipeline = (
            f"v4l2src device=/dev/video{self.camera_index} ! "
            f"video/x-raw,width={self.width},height={self.height},framerate={self.framerate}/1 ! "
            "videoconvert ! "
            "appsink"
        )
        logger.debug(f"Pipeline de GStreamer: {pipeline}")
        return pipeline

    def initialize_camera(self) -> bool:
        """
        Inicializa la cámara con el pipeline de GStreamer.

        Returns:
            bool: True si la inicialización es exitosa, False en caso contrario.
        """
        try:
            pipeline = self.create_gstreamer_pipeline()
            self.cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)
            if not self.cap.isOpened():
                logger.error("No se pudo abrir la cámara.")
                return False
            logger.info("Cámara inicializada correctamente.")
            return True
        except Exception as e:
            logger.error(f"Error al inicializar la cámara: {str(e)}")
            return False

    def display_feed(self):
        """Captura y muestra el video de la cámara en una ventana."""
        try:
            cv2.namedWindow(self.window_name, cv2.WINDOW_AUTOSIZE)
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    logger.warning("No se pudo leer el frame de la cámara.")
                    break

                # Mostrar el frame en la ventana
                cv2.imshow(self.window_name, frame)

                # Presionar 'q' para salir
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    logger.info("Cámara detenida por el usuario.")
                    break

        except Exception as e:
            logger.error(f"Error durante la captura de video: {str(e)}")
        finally:
            self.cleanup()

    def cleanup(self):
        """Libera los recursos de la cámara y cierra las ventanas."""
        if self.cap is not None and self.cap.isOpened():
            self.cap.release()
            logger.info("Recursos de la cámara liberados.")
        cv2.destroyAllWindows()
        logger.info("Ventanas cerradas.")

    def run(self):
        """Método principal para ejecutar el streamer de la cámara."""
        if not self.initialize_camera():
            logger.error("No se pudo iniciar el programa.")
            sys.exit(1)
        self.display_feed()

def main():
    """Función principal para ejecutar el programa."""
    try:
        # Configuración de la cámara (puedes ajustar estos parámetros)
        streamer = CameraStreamer(
            camera_index=0,
            width=1280,
            height=720,
            framerate=30
        )
        streamer.run()
    except KeyboardInterrupt:
        logger.info("Programa terminado por el usuario.")
    except Exception as e:
        logger.error(f"Error en el programa: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()

ERROR:__main__:No se pudo abrir la cámara.
ERROR:__main__:No se pudo iniciar el programa.


SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
